In [198]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, InputLayer
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.utils import to_categorical
from keras.optimizers import Adam
import cv2
import numpy as np
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

In [199]:
def makeImageArray(directory): 
    imgs = []
    for i in sorted(os.listdir(directory)): 
        path = os.path.join(directory, i)
        img = cv2.imread(path, cv2.COLOR_BGR2RGB)
        try: 
            img = cv2.resize(img, (128,128))
            imgs.append(img)
        except: 
            pass #jupyter hidden file throwing an error 
    return np.array(imgs)

In [200]:
def makeModelArchitecture(): 
    model = Sequential()
    model.add(InputLayer(input_shape=[128,128,3]))
    
    model.add(Conv2D(filters=32, kernel_size=5, strides=1, padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=5, padding='same'))

    model.add(Conv2D(filters=50, kernel_size=5, strides=1, padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=5, padding='same'))

    model.add(Conv2D(filters=80, kernel_size=5, strides=1, padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=5, padding='same'))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    optimizer = Adam(lr=1e-3)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [201]:
def convertToCategorical(y): 
    return to_categorical(y, 4)

In [202]:
def fitModel(model, train_X, train_y): 
    model.fit(x=train_X, y=train_y, epochs=50, batch_size=100)

In [209]:
def makePredictionsAndComputeAccuracy(test_X, test_y):
    predictions =[]
    results = model.predict(test_X)
    for r in results: 
        maxval = np.max(r)
        predictions.append(np.squeeze(np.where(r==maxval)[0]).item(0))
    print(len([i for i, j in zip(predictions, test_y) if i == j])/len(test_y))

In [204]:
train_dir = 'data/images/train'
train_y_sparse = [3,3,3,3,1,0,1,0,1,1,0,1,1,0,0,1,1,0,1,1,0,0,1,2,0,1,1,0,1,1,1,2,0,0,0,0,0,1,1,0,0,1,1,0,1,1,0,0,1,1,1,1,1,0,0,1,1,0,1,1,1,1,0,0,1,0,0,1,1,1,1,0,0,1,1,1,0,1,1,1,2,0,1,1,1,1,1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,0,1,1]
test_dir = 'data/images/test'
test_y_sparse = [3,1,1,1,0,0,0,1,1,1,0]

train_X = makeImageArray(train_dir)
test_X = makeImageArray(test_dir)

train_y = convertToCategorical(train_y_sparse)
test_y = convertToCategorical(test_y_sparse)

model = makeModelArchitecture()
fitModel(model, train_X, train_y)

Epoch 1/50
104/104 [==============================] - 1s 11ms/step - loss: 46.0205 - accuracy: 0.2212
Epoch 2/50
104/104 [==============================] - 1s 9ms/step - loss: 28.0913 - accuracy: 0.4808
Epoch 3/50
104/104 [==============================] - 1s 11ms/step - loss: 21.8226 - accuracy: 0.3942
Epoch 4/50
104/104 [==============================] - 1s 9ms/step - loss: 13.6850 - accuracy: 0.4423
Epoch 5/50
104/104 [==============================] - 1s 9ms/step - loss: 5.8735 - accuracy: 0.6058
Epoch 6/50
104/104 [==============================] - 1s 9ms/step - loss: 5.4297 - accuracy: 0.5865
Epoch 7/50
104/104 [==============================] - 1s 9ms/step - loss: 4.2846 - accuracy: 0.4327
Epoch 8/50
104/104 [==============================] - 1s 9ms/step - loss: 3.0924 - accuracy: 0.5000
Epoch 9/50
104/104 [==============================] - 1s 9ms/step - loss: 1.7424 - accuracy: 0.5577
Epoch 10/50
104/104 [==============================] - 1s 9ms/step - loss: 1.4270 - accuracy: 

In [205]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 128, 128, 32)      2432      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 26, 26, 50)        40050     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 6, 6, 50)          0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 6, 6, 80)          100080    
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 2, 2, 80)          0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 2, 2, 80)        

In [210]:
makePredictionsAndComputeAccuracy(test_X, test_y_sparse)

0.7272727272727273


In [188]:
choices =['front', 'inside', 'house_other', 'not_house']

In [189]:
def humanReadableLabel(prediction_lbl): 
    return choices[prediction_lbl]